In [11]:
from yahooquery import Ticker
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
ticker_name = 'PLTR'
ticker = Ticker(ticker_name)
df = ticker.history().loc[ticker_name]

c:\Users\basel\Projects\options-app\venv\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")


In [16]:
returns_on_interval = ((df['close'].shift(-1) - df['close'])/df['close'].shift(-1)).dropna()
fig = px.histogram(returns_on_interval)
fig.update_layout(bargap=0.05)

In [15]:
px.line(returns_on_interval)

In [4]:
fig = go.Figure(
    data = [
        go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close']
            )
    ]
)
fig.show()